## Markov Decision Process: Monte Carlo Method

In [ ]:
import numpy as np

# state tranision matrix
P = np.array([
  [0.5, 0.5, 0.0, 0.0, 0.0],
  [0.5, 0.0, 0.5, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.5, 0.5],
  [0.0, 0.1, 0.2, 0.2, 0.5],
  [0.0, 0.0, 0.0, 0.0, 0.0], # 终点
])

# reward matrix, -100表示不可能走到
R = np.array([
  [-1.0, 0.0, -100.0, -100.0, -100.0],
  [-1.0, -100.0, -2.0, -100.0, -100.0],
  [-100.0, -100.0, -100.0, -2.0, 0.0],
  [-100.0, 1.0, 1.0, 1.0, 10.0],
  [-100.0, -100.0, -100.0, -100.0, -100.0],
])

P, R

(array([[0.5, 0.5, 0. , 0. , 0. ],
        [0.5, 0. , 0.5, 0. , 0. ],
        [0. , 0. , 0. , 0.5, 0.5],
        [0. , 0.1, 0.2, 0.2, 0.5],
        [0. , 0. , 0. , 0. , 0. ]]),
 array([[  -1.,    0., -100., -100., -100.],
        [  -1., -100.,   -2., -100., -100.],
        [-100., -100., -100.,   -2.,    0.],
        [-100.,    1.,    1.,    1.,   10.],
        [-100., -100., -100., -100., -100.]]))

In [ ]:
import random

# 生成一个chain
def get_chain(max_lens):
  # 采样结果
  ss = []
  rs = []
  
  # 随机选择一个非终点（4号状态）作为起点
  s = random.choice(range(4))
  ss.append(s)
  
  for _ in range(max_lens):
    # 按照P的概率，找到下一个状态, 最多走max_lens步
    s_next = np.random.choice(np.arange(5), p=P[s])
    
    # 取到r
    r = R[s, s_next]
    
    # s_next 变为当前状态，开始接下来的循环
    s = s_next
    
    ss.append(s)
    rs.append(r)
    
    # 如果到了状态4，则结束
    if s == 4:
      break
    
  return ss, rs

get_chain(20)

([2, 3, 2, 3, 4], [-2.0, 1.0, -2.0, 10.0])

In [5]:
# 生成N个chain
def get_chains(N, max_lens):
  ss = []
  rs = []
  
  for _ in range(N):
    s, r = get_chain(max_lens)
    ss.append(s)
    rs.append(r)
    
  return ss, rs

ss, rs = get_chains(100, 20)

ss, rs

([[2, 4],
  [2, 3, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 2, 4],
  [2, 4],
  [2, 4],
  [2, 4],
  [2, 4],
  [3, 4],
  [3, 2, 4],
  [1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 4],
  [1, 2, 3, 4],
  [2, 4],
  [0, 0, 1, 0, 0, 1, 0, 1, 2, 4],
  [2, 4],
  [3, 2, 3, 1, 2, 4],
  [0, 1, 0, 1, 2, 3, 4],
  [1, 0, 0, 0, 1, 0, 0, 1, 2, 3, 2, 4],
  [2, 3, 4],
  [2, 3, 4],
  [2, 3, 2, 4],
  [1, 2, 3, 3, 1, 2, 4],
  [3, 2, 4],
  [1, 2, 4],
  [0, 0, 1, 2, 3, 4],
  [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
  [1, 0, 1, 2, 3, 4],
  [2, 4],
  [3, 4],
  [3, 4],
  [3, 4],
  [2, 3, 4],
  [2, 4],
  [2, 3, 2, 4],
  [0, 1, 0, 0, 1, 0, 1, 2, 3, 4],
  [3, 3, 4],
  [0, 0, 1, 2, 3, 2, 4],
  [2, 3, 4],
  [0, 0, 0, 1, 2, 4],
  [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 4],
  [0, 0, 1, 0, 0, 1, 2, 4],
  [3, 4],
  [3, 2, 3, 2, 3, 2, 3, 2, 3, 4],
  [0, 0, 0, 1, 2, 3, 4],
  [2, 3, 4],
  [2, 3, 4],
  [2, 4],
  [1, 0, 0, 1, 2, 4],
  [0, 1, 0, 0, 1, 2, 3, 4],
  [3, 1, 2, 3, 4],
  [3, 2, 4],
  [1, 

In [6]:
# 给定链，计算回报
def get_value(rs):
  sum = 0
  for i, r in enumerate(rs):
    # discounted return
    sum += 0.5 ** i * r 
    
  # 返回‘return’ 
  return sum

get_value(rs[0])

0.0

In [7]:
# Monte Carlo 评估每个状态的价值
def get_values_mc(ss, rs):
  # 记录4个不同开头的价值 （4号不能作为开头）
  values = [[] for _ in range(5)]
  
  # 遍历所有链
  for s, r in zip(ss, rs):
    # 计算不同开头的价值
    values[s[0]].append(get_value(r))
    
  # 求每个开头的平均值
  return [np.mean(i) for i in values]
  
get_values_mc(*get_chains(2000, 20))

c:\Users\fanyu\miniconda3\envs\RL_learning2\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\fanyu\miniconda3\envs\RL_learning2\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[-1.2186738165739541,
 -1.7287535093545392,
 0.5724098329369969,
 6.161147634265492,
 nan]

In [8]:
# 计算状态动作对(s, a)出现的频率，以此来估算策略的“占用度量”
def occupancy(ss, rs, s, a):
  rho = 0
  
  count_by_time = np.zeros(max_time)
  count_by_s_a = np.zeros(max_time)
  
  for si, ri in zip(ss, rs):
    for i in range(len(ri)):
      s_opt = si[i]
      a_opt = si[i + 1]
      
      # 统计每个时间步的次数
      count_by_time[i] += 1
      
      # 统计s, a出现的次数
      if s == s_opt and a == a_opt:
        count_by_s_a[i] += 1
        
  # i -> [999 - 0]
  for i in reversed(range(max_time)):
    if count_by_time[i] == 0:
      continue
    
    # 时间衰减
    rho += 0.5 ** i * count_by_s_a[i] / count_by_time[i]
    
  return (1 - 0.5) * rho

max_time = 1000
ss, rs = get_chains(max_time, 2000)

occupancy(ss, rs, 3, 1) + occupancy(ss, rs, 3, 2) + occupancy(ss, rs, 3, 3)

0.11229629064420303

In [10]:
# redefine transition probability matrix
P = np.array([
  [0.6, 0.4, 0.0, 0.0, 0.0],
  [0.3, 0.0, 0.7, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.5, 0.5],
  [0.0, 0.18, 0.36, 0.36, 0.1],
  [0.0, 0.0, 0.0, 0.0, 0.0],  
])

ss, rs = get_chains(max_time, 2000)
occupancy(ss, rs, 3, 1) + occupancy(ss, rs, 3, 2) + occupancy(ss, rs, 3, 3)


0.2241699249260461